In [1]:
!pip install tensorflow keras matplotlib opencv-python kaggle

In [2]:
from google.colab import files
files.upload()  # a file browser will pop up → select your kaggle.json



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kirantl23btai048","key":"1ae106e3310513d8846ab50707485661"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip -q chest-xray-pneumonia.zip -d data


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
100% 2.29G/2.29G [00:15<00:00, 106MB/s] 
100% 2.29G/2.29G [00:15<00:00, 156MB/s]


In [5]:
train_dir = "data/chest_xray/train"
val_dir   = "data/chest_xray/val"
test_dir  = "data/chest_xray/test"


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

# Validation and test data only rescaled
val_datagen = ImageDataGenerator(rescale=1./255)

# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Validation generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build CNN
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # binary classification: Pneumonia / Normal
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# See model summary
model.summary()



/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
history = model.fit(
    train_generator,
    epochs=5,  # hackathon demo: enough to see results
    validation_data=val_generator
)



/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step - accuracy: 0.7680 - loss: 0.5554

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 79s 444ms/step - accuracy: 0.7682 - loss: 0.5547 - val_accuracy: 0.6875 - val_loss: 0.6793
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 75s 430ms/step - accuracy: 0.8789 - loss: 0.2861 - val_accuracy: 0.6250 - val_loss: 1.1714
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 71s 433ms/step - accuracy: 0.8829 - loss: 0.2598 - val_accuracy: 0.6875 - val_loss: 0.5645
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 72s 443ms/step - accuracy: 0.9105 - loss: 0.2166 - val_accuracy: 0.6250 - val_loss: 1.1532
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 70s 431ms/step - accuracy: 0.9233 - loss: 0.1966 - val_accuracy: 0.5625 - val_loss: 1.0923


In [9]:
model.save("/content/drive/MyDrive/Colab Notebooks/chest_xray/pneumonia_model.h5")

In [15]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image

# Load the saved model
model_path = "/content/drive/MyDrive/Colab Notebooks/chest_xray/pneumonia_model.h5"
model = load_model(model_path)

# Class names
class_names = ["Normal", "Pneumonia"]

# Image preprocessing function
def predict_pneumonia(img):
    # Ensure the image is RGB
    if img.mode != "RGB":
        img = img.convert("RGB")

    # Resize to model input size (adjust if your model input is different)
    img = img.resize((150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize

    # Predict
    prediction = model.predict(img_array)[0][0]

    # If model outputs a single neuron with sigmoid
    if prediction > 0.5:
        result = "Pneumonia"
        confidence = prediction
    else:
        result = "Normal"
        confidence = 1 - prediction

    return {result: float(confidence)}

# Gradio interface
title = "Chest X-Ray Pneumonia Detection 🫁"
description = "Upload a chest X-ray image, and the model will predict whether it's **Normal** or **Pneumonia**. The model was trained on real chest X-ray images."

interface = gr.Interface(
    fn=predict_pneumonia,
    inputs=gr.Image(type="pil", label="Upload Chest X-Ray"),
    outputs=gr.Label(num_top_classes=2, label="Prediction"),
    title=title,
    description=description,
    theme="default",
    allow_flagging="never",
)

# Launch the app
interface.launch()


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://062607447c900aba78.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Preprocess test data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    "data/chest_xray/test",
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)

# Evaluate
loss, acc = model.evaluate(test_generator)
print(f"✅ Test Accuracy: {acc*100:.2f}%")


Found 624 images belonging to 2 classes.
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 556ms/step - accuracy: 0.8926 - loss: 0.3467
✅ Test Accuracy: 87.82%


In [ ]:
!ls data/chest_xray/test/NORMAL
!ls data/chest_xray/test/PNEUMONIA



IM-0001-0001.jpeg	   NORMAL2-IM-0173-0001-0001.jpeg
IM-0003-0001.jpeg	   NORMAL2-IM-0173-0001-0002.jpeg
IM-0005-0001.jpeg	   NORMAL2-IM-0195-0001.jpeg
IM-0006-0001.jpeg	   NORMAL2-IM-0196-0001.jpeg
IM-0007-0001.jpeg	   NORMAL2-IM-0198-0001.jpeg
IM-0009-0001.jpeg	   NORMAL2-IM-0199-0001.jpeg
IM-0010-0001.jpeg	   NORMAL2-IM-0201-0001.jpeg
IM-0011-0001-0001.jpeg	   NORMAL2-IM-0206-0001.jpeg
IM-0011-0001-0002.jpeg	   NORMAL2-IM-0207-0001.jpeg
IM-0011-0001.jpeg	   NORMAL2-IM-0210-0001.jpeg
IM-0013-0001.jpeg	   NORMAL2-IM-0213-0001.jpeg
IM-0015-0001.jpeg	   NORMAL2-IM-0217-0001.jpeg
IM-0016-0001.jpeg	   NORMAL2-IM-0219-0001.jpeg
IM-0017-0001.jpeg	   NORMAL2-IM-0221-0001.jpeg
IM-0019-0001.jpeg	   NORMAL2-IM-0222-0001.jpeg
IM-0021-0001.jpeg	   NORMAL2-IM-0229-0001.jpeg
IM-0022-0001.jpeg	   NORMAL2-IM-0232-0001.jpeg
IM-0023-0001.jpeg	   NORMAL2-IM-0233-0001.jpeg
IM-0025-0001.jpeg	   NORMAL2-IM-0237-0001.jpeg
IM-0027-0001.jpeg	   NORMAL2-IM-0238-0001.jpeg
IM-0028-0001.jpeg	   NORMAL2-IM-0241-000

In [9]:
img_path = "/content/drive/MyDrive/Colab Notebooks/chest_xray/chest_xray/test/PNEUMONIA/person100_bacteria_477.jpeg"


In [ ]:
import os

test_normal = "data/chest_xray/test/NORMAL"
for img_file in os.listdir(test_normal)[:5]:  # test first 5 images
    img_path = os.path.join(test_normal, img_file)
    img = image.load_img(img_path, target_size=(150,150))
    img_array = np.expand_dims(image.img_to_array(img)/255., axis=0)
    prediction = model.predict(img_array)
    print(img_file, "->", "Pneumonia" if prediction[0][0]>0.5 else "Normal")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
IM-0061-0001.jpeg -> Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
IM-0111-0001.jpeg -> Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
NORMAL2-IM-0301-0001.jpeg -> Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
NORMAL2-IM-0095-0001.jpeg -> Pneumonia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
NORMAL2-IM-0351-0001.jpeg -> Normal
